### Media-buying optimization with linear programming using docplex

Media planners, as their title suggests, plan advertising placements for their campaigns across a variety of different platforms. This planning involves a certain amount of media-buying, wherein third party media must be purchased according to the demands of the campaign. This is an incredibly messy process, with media planners often unable to sort through and adequately compare different publishers' offerings for sponsored content, such as articles or videos. 

In this typical example, a media planner is given constraints by their organization. They need to hit two audiences served by two different publications.

Assume Publication 1 {pub1} is a B2B-focused technology news platform that serves the Southeast Asia market, and Publication 2 {pub2} is a B2B-focused technology news platform that serves the ANZ market. 

Publication 1 receives higher viewership at lower cost, but serves a less valuable customer for readers of its branded content (measured in the click-through rates provided). Publication 2 receives lower viewership at higher cost, but serves a more valuable customer as defined by click-through numbers.

The task of the media planner is to maximize their given budget ($30,000) and allocate it efficiently across the two publications.

### Import the docplex Model

In [1]:
from docplex.mp.model import Model

We will use docplex and its package 'model' to maximize the values we want within the constraints provided by the media planner's organization

In [2]:
m_views=Model(name='View Optimizer_Views')

### Defining the variables

Assume pub1 and pub2 represent two non-overlapping audiences we want to hit.

We will use a sample product from each publication - usually a standard 'sponsored article' also known as an 'advertorial'.

We need to find out the recommended proportion of articles from each publication to maximize viewership and our given budget.


In [3]:
#items variables
pub1 = m_views.continuous_var(name='publication 1 no. of articles')
pub2 = m_views.continuous_var(name='publication 2 no. of articles')

Here we define our continous variables as the number of articles we will requisition from each of the two publishers

In [4]:
#costs per article
pub1costs = 5000
pub2costs = 6000

The costs for one article are given in dollars here from each publication respectively

In [5]:
#views variable
pub1views= 7000
pub2views= 3000

The views of each publication (assuming non-overlapping) will be compared against cost

We will also need to justify the constraints by calculating expected conversion later on

### Constraints

In [6]:
#assume client wants at least one of each, without decimal
pub1constraint= m_views.add_constraint(pub1>=2)
pub2constraint= m_views.add_constraint(pub2>=1)

The constraints provided by media planner's organization request at least 14000 views from Southeast Asia and 3000 from ANZ, based on their own internal calculations of conversion value

In [7]:
#costs constraint, say 30,000
budget = 30000
totalcostconstraint=m_views.add_constraint(m_views.sum([pub1*pub1costs,pub2*pub2costs])<=budget)

This is the total constraint value being loaded into a linear programming format

### Use the maximize function to solve the equation

In [8]:
#Maximize function
m_views.maximize(pub1*pub1views+pub2*pub2views)


Our value 'm' is now passed through the docplex maximize function with its original variables, the constraints are located in totalcostconstraint

In [9]:
solver_views=m_views.solve()

In [10]:
solver_views.display()

solution for: View Optimizer_Views
objective: 36600.000
publication 1 no. of articles = 4.800
publication 2 no. of articles = 1.000


The solver overwhelming prefers publication 1, this is likely due to the view difference between the two publications being too wide

### Find the expected results of the campaign by summing the outputs

In [11]:
#convert to integers since we can usually only purchase whole articles
pub1_max_articles = int(solver_views.get_value(pub1))
pub2_max_articles = int(solver_views.get_value(pub2))

pub1_spend = pub1_max_articles * pub1costs
pub2_spend = pub2_max_articles * pub2costs

In [12]:
total_spend = pub1_spend + pub2_spend
total_views = (pub1_max_articles * pub1views) + (pub2_max_articles * pub2views)
remainder = budget - total_spend

In [13]:
print(f"Total spend: {total_spend}, Total views: {total_views}, Remainder:{remainder}")

Total spend: 26000, Total views: 31000, Remainder:4000


The budget is finalized with 4 articles from Publication 1 and 1 from Publication 2 with a remainder of $3000 - this is not great for the media planner. They need to use their budget or lose it. 

Assuming our media planner is content with increasing their per-dollar cost for views, we could increase the spend on Publication 2. To make this decision, we must infer the expected ROI from each publication in a seperate solver.

### Repeat for an ROI (conversion) focused comparison

In [14]:
#ROI variables
pub1_conversions_per_article = 70
pub2_conversions_per_article = 100

Assume that each publisher is able to give a reasonable expectation of click-throughs (this is pure fantasy, as no publisher would ever reveal these stats due to their abysmal nature) designated as cost-per-click (CPC) onwards to the media planner's campaign.

We also want to retain our minimum constraints, given we still have a minimnum view target to hit. There is a difference of 50 conversions (click-throughs) between articles. We are choosing to represent this number as per article rather than per view, given that this is how these numbers are actually deduced on the publisher side.

In [15]:
m_ROI=Model(name='CPC Optimizer_ROI')
#New model for new constraints and variables
pub1_ROI = m_ROI.continuous_var(name='publication 1 no. of articles ROI')
pub2_ROI = m_ROI.continuous_var(name='publication 2 no. of articles ROI')

In [16]:
pub1constraint_ROI= m_ROI.add_constraint(pub1_ROI>=2)
pub2constraint_ROI= m_ROI.add_constraint(pub2_ROI>=1)
#Remove minimum constraints
totalcostconstraint_ROI=m_ROI.add_constraint(m_ROI.sum([pub1_ROI*pub1costs,pub2_ROI*pub2costs])<=30000)
m_ROI.maximize(pub1*pub1_conversions_per_article+pub2*pub2_conversions_per_article)

In [17]:
solver_ROI=m_ROI.solve()

In [18]:
solver_ROI.display()

solution for: CPC Optimizer_ROI
objective: 473.333
publication 1 no. of articles ROI = 2.000
publication 2 no. of articles ROI = 3.333


In [19]:
pub1_ROI_optimized=int(solver_ROI.get_value(pub1_ROI))
pub2_ROI_optimized=int(solver_ROI.get_value(pub2_ROI))
pub1_spend_ROI = pub1_ROI_optimized * pub1costs
pub2_spend_ROI = pub2_ROI_optimized * pub2costs

In [20]:
total_spend2 = pub1_spend + pub2_spend
### We can also calculate total conversions for the first method - (maximizing by views) here
total_conversions = (pub1_max_articles * pub1_conversions_per_article) + (pub2_max_articles * pub2_conversions_per_article)
total_conversions2 = (pub1_ROI_optimized * pub1_conversions_per_article) + (pub2_ROI_optimized * pub2_conversions_per_article)
total_views2 = (pub1_ROI_optimized * pub1views) + (pub2_ROI_optimized * pub2views)
remainder2 = budget - total_spend

In [21]:
print(f"View-maximizing solver -> Total spend: {total_spend}, Total views: {total_views}, Total conversions: {total_conversions}, Remainder:{remainder}")
print(f"ROI-maximizing solver -> Total spend: {total_spend2}, Total views: {total_views2}, Total conversions: {total_conversions2}, Remainder:{remainder2}")

View-maximizing solver -> Total spend: 26000, Total views: 31000, Total conversions: 380, Remainder:4000
ROI-maximizing solver -> Total spend: 26000, Total views: 23000, Total conversions: 440, Remainder:4000


### Wa-wa-wee-wah

Interestingly, the solver returns a much lower value for articles from Publication 1 - it is the mininmum required. The slightly higher value of 30 more conversions per article for Publication 2 has resulted in a total conversion difference of 60 more, with a penalty of 8000 less views. Although minimums are hit, it is always difficult to justify the difference between the magnitude of these two numbers. 8000 more views or 60 more conversions? Media planners are often prone to this kind of magnitude fixation. The next step would be to see which of the two publishers have click-throughs that are turning into a conversion. One enterprise-level conversion can practically cover the expense of this campaign and then some in this technology industry.

However, we are still left with $3000 remainder. This cannot really be avoided given the difficulty of fitting the prices for each publisher inside the stipulated budget.

The lesson here is two fold:

1) Decide if your campaign is going to focus on conversion or awareness - while you can do both within the same campaign, publisher products may not be optimized for such across different markets. It is ultimately a little fruitless to compare views across vastly different geos if they display a pattern that inspired our View-maximizing solver. Conversions are likely more valuable for any performance marketer, so it would be pertinent to decide amongst those two early, especially in this B2B example.
 
2) Always ask for budget after running an optimization...


To learn more about optimizing media budgets when selecting publishers, message me at jamin@campaignr.biz